In [1]:
from AnthropicWrapper import ClaudeChat
from AnthropicWrapper import Utilities
from joblib import load
import os
import fitz
import numpy as np
from hume_analysis import HumeSentimentAnalyzer
from dotenv import load_dotenv

In [2]:
# --- LLM-2 Configuration ---
chat_model_name = "claude-3-5-sonnet-20240620"
llm2_system_prompt = """
You are a seasoned hiring manager evaluating a candidate for a [Job Title] position at [Company Name]. The candidate has [Experience Level] experience in the field. You have access to the candidate's CV, the transcript of their interview, and sentiment analysis data from Hume.ai.
You are to primarily use the transcript of the interview to make your decision on the candidate's performance. Further supportive information can be found in the sentiment analysis and CV to assist, but the primary analysis should lie with the transcript.
Please take into account in order the accuracy of the candidates answer, sentimnet and then the others skills mentioned here.

**Task:**

1. **Understand the Job Requirements:** Carefully analyze the [Job Description] provided and identify the key skills, experience, and qualifications required for success in this role.

2. **Evaluate Candidate Alignment:** Assess how well the candidate's qualifications and experience align with the identified job requirements.
    * **Accuracy** Based on your knowledge please analyze the accuracy of the candidates answer. Give an accuracy rating i percentage for the candidate.
    * **Technical Skills:**  Does the candidate demonstrate the necessary technical skills, including specific software, tools, and frameworks mentioned in the job description? 
    * **Project Experience:**  Are the candidate's projects relevant to the role and demonstrate the required level of complexity and problem-solving abilities?
    * **Soft Skills:**  Does the candidate possess the necessary communication, collaboration, and problem-solving skills?
    * **Cultural Fit:**  Does the candidate seem to align with the company's values and work environment as described in the job description?

3. **Analyze Sentiment Data:**  Use the sentiment analysis data from Hume.ai to understand the candidate's emotional state during the interview.  Does their emotional response indicate confidence, enthusiasm, or any potential issues?

4. **Analyze Toxicity Scores:**  Does the candidate's communication exhibit any signs of toxicity, unprofessionalism, or potentially problematic behavior?

5. **Provide a Recommendation:** 
    * **Strong Candidate:** The candidate demonstrates strong technical skills, relevant experience, and a positive attitude. Recommend moving them forward in the interview process.
    * **Consider for Further Evaluation:** The candidate has potential, but their technical skills or project experience need further investigation. Recommend additional interviews or assessments. 
    * **Not a Fit:** The candidate lacks the necessary technical skills or experience, or their communication/attitude was not compelling. Recommend not moving forward with this candidate.

**Explain your reasoning:** Provide specific evidence from the interview transcript, sentiment analysis of the interview, and CV to support your recommendation. 
"""


In [3]:
role_description = """
Do you want to tackle the biggest questions in finance with near infinite compute power at your fingertips?

G-Research is a leading quantitative research and technology firm, with offices in London and Dallas. We are proud to employ some of the best people in their field and to nurture their talent in a dynamic, flexible and highly stimulating culture where world-beating ideas are cultivated and rewarded.

This is a role based in our new Soho Place office - opened in 2023 - in the heart of Central London and home to our Research Lab.

The role

We are looking for exceptional machine learning engineers to work alongside our quantitative researchers on cutting-edge machine learning problems.

As a member of the Core Technical Machine Learning team, you will be engaged in a mixture of individual and collaborative work to tackle some of the toughest research questions.

In this role, you will use a combination of off-the-shelf tools and custom solutions written from scratch to drive the latest advances in quantitative research.

Past projects have included:

Implementing ideas from a recently published research paper
Writing custom libraries for efficiently training on petabytes of data
Reducing model training times by hand optimising machine learning operations
Profiling custom ML architectures to identify performance bottlenecks
Evaluating the latest hardware and software in the machine learning ecosystem
Who are we looking for?

Candidates will be comfortable working both independently and in small teams on a variety of engineering challenges, with a particular focus on machine learning and scientific computing.

The ideal candidate will have the following skills and experience:

Either a post-graduate degree in machine learning or a related discipline, or commercial experience working on machine learning models at scale. We will also consider exceptional candidates with a proven record of success in online data science competitions, such as Kaggle
Strong object-oriented programming skills and experience working with Python, PyTorch and NumPy are desirable
Experience in one or more advanced optimisation methods, modern ML techniques, HPC, profiling, model inference; you dont need to have all of the above
Excellent ML reasoning and communication skills are crucial: off-the-shelf methods dont always work on our data so you will need to understand how to develop your own models in a collaborative environment working in a team with complementary skills
Finance experience is not necessary for this role and candidates from non-financial backgrounds are encouraged to apply.

Why should you apply?

Highly competitive compensation plus annual discretionary bonus
Lunch provided (via Just Eat for Business) and dedicated barista bar
35 days annual leave
9 percent company pension contributions
Informal dress code and excellent work/life balance
Comprehensive healthcare and life assurance
Cycle-to-work scheme
Monthly company events
"""

In [4]:
# --- Function for LLM-2 analysis ---
def analyze_candidate(job_title, experience_level, job_description, interview_transcript, cv, analyzer_output):
    """
    Analyzes a candidate's suitability based on the provided data.

    Args:
        job_title (str): The title of the job.
        experience_level (str): The candidate's experience level.
        job_description (str): The text of the job description.
        interview_transcript (str): The transcript of the interview.
        cv (str): The candidate's CV.
        sentiment_data (dict): Sentiment data from Hume.ai.
        toxicity_data (dict): Toxicity data from Hume.ai.

    Returns:
        str: The recommendation and reasoning from LLM-2.
    """

    # Create a prompt with placeholders for dynamic values
    llm2_prompt = llm2_system_prompt.replace("[Job Title]", job_title)
    llm2_prompt = llm2_prompt.replace("[Experience Level]", experience_level)
    llm2_prompt = llm2_prompt.replace("[Job Description]", job_description)
    
     # --- Preprocess Sentiment Data ---
    positive_emotions = ["Joy", "Enthusiasm", "Excitement", "Pride", "Satisfaction", "Triumph", "Surprise (positive)", "Contentment", "Ecstasy", "Gratitude", "Love", "Relief"]
    negative_emotions = ["Anger", "Annoyance", "Anxiety", "Boredom", "Contempt", "Disappointment", "Disapproval", "Disgust", "Distress", "Fear", "Guilt", "Horror", "Pain", "Sadness", "Shame", "Surprise (negative)", "Sympathy", "Tiredness"]

    positive_sentiment_score = np.mean([analyzer_output["emotions"].get(emotion, 0) for emotion in positive_emotions])
    negative_sentiment_score = np.mean([analyzer_output["emotions"].get(emotion, 0) for emotion in negative_emotions])

    # --- Preprocess Toxicity Data ---
    toxicity_weights = {"severe_toxic": 0.3, "threat": 0.2, "toxic": 0.15, "identity_hate": 0.1, "insult": 0.1, "obscene": 0.1}
    overall_toxicity_score = np.sum([analyzer_output["toxicity"].get(category, 0) * toxicity_weights[category] for category in toxicity_weights])
    toxicity_level = "Low" if overall_toxicity_score < 0.01 else "Moderate" if overall_toxicity_score < 0.05 else "High"

    # ---  Preprocess Sentiment Scores (1-9) ---
    sentiment_scores = analyzer_output["sentiments"] 
    sentiment_scores = analyzer_output["sentiments"]
    overall_sentiment_score = sum(float(key) * float(value) for key, value in sentiment_scores.items())
    overall_sentiment_level = "Highly Negative" if overall_sentiment_score < 3 else "Negative" if overall_sentiment_score < 6 else "Neutral" if overall_sentiment_score < 7 else "Positive" if overall_sentiment_score < 8 else "Highly Positive"
    
    # --- Add Preprocessed Data to the Analyzer Output ---
    analyzer_output["Preprocessed Sentiment"] = {
        "Positive Sentiment": positive_sentiment_score,
        "Negative Sentiment": negative_sentiment_score,
        "Overall Sentiment Score (1-9)": overall_sentiment_score,
        "Overall Sentiment Level": overall_sentiment_level
    }

    analyzer_output["Preprocessed Toxicity"] = {
        "Overall Toxicity Score": overall_toxicity_score,
        "Toxicity Level": toxicity_level
    }

    # --- Prepare the data for LLM-2 ---
    analysis_data = {
        "Interview Transcript": interview_transcript,
        "CV": cv,
        "Analyzer Output": analyzer_output  # Pass the entire dictionary
    }

    llm2_prompt += "\n\nHere is the data for your analysis:\n"
    llm2_prompt += "\n".join(f"{key}: {value}" for key, value in analysis_data.items())
    global claude
    claude = ClaudeChat(chat_model_name, llm2_prompt)
    return claude.chat("Please Execute the system prompt")


In [5]:
model = "claude-3-5-sonnet-20240620"
job_role = "Machine Learning Engineer"
candidate_skill = "Entry-Level"
utilities = Utilities()
transcript_path = "conversation.pdf"
transcript = utilities.process_pdf(transcript_path)
pdf_path = r"D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"
cv = utilities.process_pdf(transcript_path)

In [6]:
"""
def extract_transcript_from_pdf(pdf_path):

    doc = fitz.open(pdf_path)
    extracted_transcript = ""

    for page_num in range(doc.page_count):
        page = doc[page_num]
        blocks = page.get_text("blocks")

        for block in blocks:
            text = block[4].strip()
            if text:
                if block[0] < 200 and block[3] > 12:  # Check for headings or dialogue lines
                    extracted_transcript += f"\n\n{text}\n"
                else:
                    extracted_transcript += f"{text} "

    return extracted_transcript
"""

'\ndef extract_transcript_from_pdf(pdf_path):\n\n    doc = fitz.open(pdf_path)\n    extracted_transcript = ""\n\n    for page_num in range(doc.page_count):\n        page = doc[page_num]\n        blocks = page.get_text("blocks")\n\n        for block in blocks:\n            text = block[4].strip()\n            if text:\n                if block[0] < 200 and block[3] > 12:  # Check for headings or dialogue lines\n                    extracted_transcript += f"\n\n{text}\n"\n                else:\n                    extracted_transcript += f"{text} "\n\n    return extracted_transcript\n'

In [7]:
# Get the directory containing the script
script_directory = os.getcwd()
print("Script Directory : "+script_directory)
# Navigate one folder up
one_folder_up = os.path.dirname(script_directory)
# Construct the path to the .env file in the parent folder
parent_folder_path = os.path.dirname(os.getcwd())
dotenv_path = os.path.join(parent_folder_path, ".env")
# Load the .env file
load_dotenv(dotenv_path)
analyzer = HumeSentimentAnalyzer(os.getenv("HUME_API_KEY"))
result = analyzer.analyze_audio(os.path.join(script_directory, "interview_audio.wav"))


Script Directory : d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\LLM2-Prompting
Analyzing audio...


In [8]:
print(result)

{'emotions': {'Admiration': 0.026119697839021683, 'Adoration': 0.003861038014292717, 'Aesthetic Appreciation': 0.035107363015413284, 'Amusement': 0.03813358396291733, 'Anger': 0.00038087990833446383, 'Annoyance': 0.010568605735898018, 'Anxiety': 0.01789218746125698, 'Awe': 0.012385101988911629, 'Awkwardness': 0.010339461266994476, 'Boredom': 0.022676732391119003, 'Calmness': 0.059521790593862534, 'Concentration': 0.3941130042076111, 'Confusion': 0.041417255997657776, 'Contemplation': 0.3485284149646759, 'Contempt': 0.010550647974014282, 'Contentment': 0.02668710984289646, 'Craving': 0.008198815397918224, 'Desire': 0.00071975082391873, 'Determination': 0.44453132152557373, 'Disappointment': 0.008252736181020737, 'Disapproval': 0.003823148086667061, 'Disgust': 0.000525407784152776, 'Distress': 0.004626619163900614, 'Doubt': 0.08696653693914413, 'Ecstasy': 0.0022196820937097073, 'Embarrassment': 0.0017864009132608771, 'Empathic Pain': 0.0033241109922528267, 'Enthusiasm': 0.443078428506851

In [9]:
# --- Call LLM-2 for Candidate Analysis ---
print("Analyzing candidate...")
llm2_recommendation = analyze_candidate(
    job_title=job_role,
    experience_level=candidate_skill,
    job_description=role_description,
    interview_transcript=transcript,
    cv=cv,
    analyzer_output=result
)

print(f"LLM-2 Recommendation: {llm2_recommendation}")

Analyzing candidate...
LLM-2 Recommendation: Based on the interview transcript, sentiment analysis, and job requirements, here's my evaluation of the candidate for the Entry-Level Machine Learning Engineer position at G-Research:

1. Accuracy of Candidate's Answers: 65%
The candidate's responses show a basic understanding of machine learning concepts, but lack depth and specificity expected for this role.

2. Evaluation of Candidate Alignment:

Technical Skills: 
The candidate demonstrates basic knowledge of Python libraries like NumPy and Pandas, which is a good starting point. However, they don't mention experience with PyTorch, which is specifically desired for this role. Their project experience is limited to basic CNN implementation on the MNIST dataset, which is a standard beginner project.

Project Experience:
The candidate's project experience is limited and basic. While they have worked on image classification using CNNs, they don't demonstrate experience with more advanced te

In [ ]:
print(claude.chat_with_history_doc(input()))